In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lead, col, broadcast, collect_set, size, array_contains
class Transformer:

    def __init__(self):
        pass

    def transform(self,inputDF):
        pass

class AirpodaAfterIphone(Transformer):

    def transform(self,inputDF):
        transactionInputDF = inputDF.get("transactionInputDF")
        # transactionInputDF.show(5)
        window = Window.partitionBy("customer_id").orderBy("transaction_date")
        transformedDF = transactionInputDF.withColumn("next_product",lead("product_name").over(window))
        # print("transformed transaction dataframe")
        # transformedDF.orderBy("customer_id","transaction_date","product_name").show(5)
        # print("transformed final transaction dataframe")
        filteredDF=transformedDF.filter((col("product_name")=="iPhone") & (col("next_product")=="AirPods"))
        # filteredDF.orderBy("customer_id","transaction_date","product_name").show(5)

        customerInputDF = inputDF.get("customerInputDF")
        #print(customerInputDF)
        # joinDF = filteredDF.join(customerInputDF,"customer_id")
        # use Broadcast Join
        #print(joinDF)
        joinDF = customerInputDF.join(broadcast(filteredDF),"customer_id")
        return joinDF.select("customer_id","customer_name","location")
    
class OnlyAirpodsAndIphone(Transformer):

    def transform(self,inputDF):
        """
        customer who have bought only IPhone and Airpods only
        """
        transactionInputDF = inputDF.get("transactionInputDF")
        groupedDF = transactionInputDF.groupBy("customer_id").agg(collect_set("product_name").alias("products"))
        groupedDF.show()

        filteredDF = groupedDF.filter( 
                                      (array_contains(col("products"),"iPhone") ) & 
                                      (array_contains(col("products"),"AirPods") ) &
                                      (size(col("products"))==2) 
                                    )
        print("filteredDF having only AirPods and IPhones")
        filteredDF.show()

        customerInputDF = inputDF.get("customerInputDF")
        # joinDF = filteredDF.join(customerInputDF,"customer_id")
        # use Broadcast Join
        #print(joinDF)
        joinDF = customerInputDF.join(broadcast(filteredDF),"customer_id")
        return joinDF.select("customer_id","customer_name","location")